## Use Habanero to Query Information on DOIs from USGS Spreadsheet

This notebook uses the Python package called [habanero](https://habanero.readthedocs.io/en/latest/modules/crossref.html) to query the [CrossRef Database API](https://github.com/CrossRef/rest-api-doc) for information on USGS DOIs for data releases and the package called [pprintjson](https://github.com/clarketm/pprintjson) for easy viewing of the query results. 

These packages can easily be installed using `pip install habanero` and `pip install pprintjson`. 

USGS has a spreadsheet that contains the data release DOI (column called dr_doi) and the DOI for the publication associated with the data release (column called rel_pub_url). 

One goal is to auto-populate the spreadsheet with the following information about the rel_pub_url:
* title
* journal name
* year of publication
* publisher

However, the primary goal is to identify whether the data release doi (dr_doi) is actually cited within the publication (rel_pub_url), and if so, document where it is cited (e.g. references section, main text body, figure caption). 

Searching of the references can be completed with habanero for those publications that have a reference list available in CrossRef. 

Per the Habanero documentation on the CrossRef API: 
> When you search with query terms, on Crossref servers, it will not search full text, or even abstracts of articles, but only what is available in the API data response options. That is, it searches titles, authors, year of publication, reference list (if one is available), etc. For some discussion on this, see https://github.com/CrossRef/rest-api-doc/issues/101

In [1]:
# Import packages
from pprintjson import pprintjson as ppjson 
import pandas as pd
from habanero import Crossref

In [2]:
# Code below adapted from:
# https://habanero.readthedocs.io/en/latest/modules/crossref.html#habanero.Crossref.works
cr = Crossref()

### Example of Publication Without Full Reference List

In [3]:
# Example publication: https://api.crossref.org/works/10.1021/acs.est.8b07227

# First row from USGS spreadsheet:
# 2019; journal title in spreadsheet: ACS Publications/Environmental Science and Technology
result = cr.works(ids = '10.1021/acs.est.8b07227')

In [4]:
# Use prettyprint to see results more easily
ppjson(result)

{
    "status": "ok",
    "message-type": "work",
    "message-version": "1.0.0",
    "message": {
        "indexed": {
            "date-parts": [
                [
                    2020,
                    4,
                    24
                ]
            ],
            "date-time": "2020-04-24T19:23:29Z",
            "timestamp": 1587756209612
        },
        "reference-count": 67,
        "publisher": "American Chemical Society (ACS)",
        "issue": "9",
        "funder": [
            {
                "DOI": "10.13039/100007149",
                "name": "U.S. Bureau of Land Management",
                "doi-asserted-by": "publisher",
                "award": []
            },
            {
                "DOI": "10.13039/100000203",
                "name": "U.S. Geological Survey",
                "doi-asserted-by": "publisher",
                "award": []
            }
        ],
        "content-domain": {
            "domain": [],
            "crossmark-restri

In [5]:
# Most relevant information contained within "message"
ppjson(result["message"])

{
    "indexed": {
        "date-parts": [
            [
                2020,
                4,
                24
            ]
        ],
        "date-time": "2020-04-24T19:23:29Z",
        "timestamp": 1587756209612
    },
    "reference-count": 67,
    "publisher": "American Chemical Society (ACS)",
    "issue": "9",
    "funder": [
        {
            "DOI": "10.13039/100007149",
            "name": "U.S. Bureau of Land Management",
            "doi-asserted-by": "publisher",
            "award": []
        },
        {
            "DOI": "10.13039/100000203",
            "name": "U.S. Geological Survey",
            "doi-asserted-by": "publisher",
            "award": []
        }
    ],
    "content-domain": {
        "domain": [],
        "crossmark-restriction": false
    },
    "short-container-title": [
        "Environ. Sci. Technol."
    ],
    "published-print": {
        "date-parts": [
            [
                2019,
                5,
                7
       

In [6]:
# Get publication DOI
result["message"]["DOI"]

'10.1021/acs.est.8b07227'

In [7]:
# Get publication date
result["message"]["published-online"]["date-parts"][0]

[2019, 3, 29]

In [1]:
# Get year from publication date
result["message"]["published-online"]["date-parts"][0][0]

NameError: name 'result' is not defined

In [9]:
# Get title of publication
result["message"]["title"][0]

'Mercury Exposure and Altered Parental Nesting Behavior in a Wild Songbird'

In [10]:
# Get publisher
result["message"]["publisher"]

'American Chemical Society (ACS)'

In [11]:
# This example contains reference count but not list of references  
result["message"]["references-count"]

67

In [12]:
# Code does not execute because this info is not available for this publication
#result["message"]["reference"]

### Example of Publication With Full Reference List

In [13]:
# Second row from USGS spreadsheet:
# 2017 journal title in spreadsheet: Changes in Community
result = cr.works(ids = 'https://doi.org/10.1007/s13157-017-0895-3')

In [14]:
# Get publication DOI
result["message"]["DOI"]

'10.1007/s13157-017-0895-3'

In [15]:
# Reference-count was accurate: 
# 89 from https://link.springer.com/article/10.1007/s13157-017-0895-3#Bib1 
result["message"]["references-count"]

89

In [16]:
# This example contains the actual list of references
# Note key for last entry in reference: "key": "895_CR89"
ppjson(result["message"]["reference"])

[
    {
        "key": "895_CR1",
        "unstructured": "Baldwin BG (2002) The Jepson Desert manual: vascular plants of southeastern California. Univ of California Press, Oakland CA"
    },
    {
        "key": "895_CR2",
        "doi-asserted-by": "crossref",
        "first-page": "1533",
        "DOI": "10.1111/j.0030-1299.2008.16776.x",
        "volume": "117",
        "author": "M Bernhardt-R\u00f6mermann",
        "year": "2008",
        "unstructured": "Bernhardt-R\u00f6mermann M, R\u00f6mermann C, Nuske R, Parth A, Klotz S, Schmidt W, Stadler J (2008) On the identification of the most suitable traits for plant functional trait analyses. Oikos 117:1533\u20131541",
        "journal-title": "Oikos"
    },
    {
        "key": "895_CR3",
        "doi-asserted-by": "crossref",
        "first-page": "287",
        "DOI": "10.1007/BF00540195",
        "volume": "45",
        "author": "TW Boutton",
        "year": "1980",
        "unstructured": "Boutton TW, Harrison AT, Smith BN (19

## For Internship Project

The USGS intern can expand on lessons on [pandas dataframes](https://www.earthdatascience.org/courses/intro-to-earth-data-science/scientific-data-structures-python/pandas-dataframes/), [loops](https://www.earthdatascience.org/courses/intro-to-earth-data-science/write-efficient-python-code/loops/), and [dictionaries](https://realpython.com/python-dicts/) ([interactive tutorial option](https://www.learnpython.org/en/Dictionaries)) to write a loop that will iterate the following code to store the information returned by the API to a new pandas dataframe. 

In [17]:
# Create empty list to store information that will be converted to dataframe
list_to_df = []

In [18]:
# Add items to list
list_to_df.append([result["message"]["DOI"],
                  result["message"]["published-online"]["date-parts"][0][0], 
                  result["message"]["title"][0],
                  result["message"]["publisher"]
                  ])

In [19]:
# Convert list to dataframe
df = pd.DataFrame(list_to_df, columns=["doi", "year", "title", "publisher"])

df

,doi,year,title,publisher
0,10.1007/s13157-017-0895-3,2017,Changes in Community-Level Riparian Plant Trai...,Springer Science and Business Media LLC


## Possible Expansion of Work

Another related aspect of this work would be to query the data release doi in [geodeepdive](https://geodeepdive.org/api/snippets?term=10.5066/F71G0JF6) and identify whether the publication listed in the spreadsheet (rel_pub_url) is associated with the data release doi and/or whether there are other publications associated wit the data release doi.

For the case of `rel_pub_url = '10.3133/sim3378'` (row 4 in the USGS spreadsheet), the publication url was found associated with the data release doi: `https://geodeepdive.org/api/snippets?term=10.5066/F71G0JF6`.